# Анализ базы данных сервиса для чтения книг по подписке

Аналитик компании, купившей крупный сервис для чтения книг по подписке, получил задачу проанализировать базу данных. В базе содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Анализ данных поможет сформулировать ценностное предложение для нового продукта.

**Задача исследования**

Проанализировать базу данных сервиса для чтения книг по подписке.

**Ход исследования**

База данных содержит 5 таблиц:

* *books* — данные о книгах;
* *authors* — данные об авторах;
* *publishers* — данные об издательствах;
* *ratings* — данные о пользовательских оценках книг;
* *reviews* — данные о пользовательских обзорах на книги.

Для начала исследуем все таблицы, выведем первые строки.

Затем, в рамках исследования, решим задачи:

* Посчитаем, сколько книг вышло после 1 января 2000 года.
* Для каждой книги посчитаем количество обзоров и среднюю оценку.
* Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.
* Определим автора с самой высокой средней оценкой книг, имеющих от 50 оценок.
* Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# Импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Задаем параметры подключения к базе данных
db_config = {
    'user': ' ', # имя пользователя
    'pwd': ' ', # пароль
    'host': ' ',
    'port': , # порт подключения
    'db': ' ' # название базы данных
}

# Формируем строку соединения с базой данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Сохраняем коннектор
engine = create_engine(
    connection_string,
    connect_args={'sslmode':'require'}
)

## Исследование таблиц
Выведем первые 5 строк каждой таблицы в базе данных.

In [3]:
# Первые 5 строк таблицы books
query = '''
        SELECT *
        FROM books
        LIMIT 5;
        '''

pd.io.sql.read_sql(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица *books* содержит данные о книгах. Согласно документации к данным:

* *book_id* — идентификатор книги;
* *author_id* — идентификатор автора;
* *title* — название книги;
* *num_pages* — количество страниц;
* *publication_date* — дата публикации книги;
* *publisher_id* — идентификатор издателя.

In [4]:
# Первые 5 строк таблицы authors
query = '''
        SELECT *
        FROM authors
        LIMIT 5;
        '''

pd.io.sql.read_sql(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица *authors* содержит данные об авторах. Согласно документации к данным:

* *author_id* — идентификатор автора;
* *author* — имя автора.

In [5]:
# Первые 5 строк таблицы publishers
query = '''
        SELECT *
        FROM publishers
        LIMIT 5;
        '''

pd.io.sql.read_sql(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица *publishers* содержит данные об издательствах .Согласно документации к данным:

* *publisher_id* — идентификатор издательства;
* *publisher* — название издательства.

In [6]:
# Первые 5 строк таблицы ratings
query = '''
        SELECT *
        FROM ratings
        LIMIT 5;
        '''

pd.io.sql.read_sql(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица *ratings* содержит данные о пользовательских оценках книг. Согласно документации к данным:

* *rating_id* — идентификатор оценки;
* *book_id* — идентификатор книги;
* username* — имя пользователя, оставившего оценку;
* *rating* — оценка книги.

In [7]:
# Первые 5 строк таблицы reviews
query = '''
        SELECT *
        FROM reviews
        LIMIT 5;
        '''

pd.io.sql.read_sql(query, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица *reviews* содержит данные о пользовательских обзорах на книги. Согласно документации к данным:

* *review_id* — идентификатор обзора;
* *book_id* — идентификатор книги;
* *username* — имя пользователя, написавшего обзор;
* *text* — текст обзора.

Содержание таблиц базы данных соответствует документации.

## Количество книг, выпущенных c 2000 года
Посчитаем, сколько книг вышло после 1 января 2000 года.

In [8]:
# Количество книг, выпущенных c 2000 года
query = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE EXTRACT(YEAR FROM publication_date) >= 2000;
        '''

pd.io.sql.read_sql(query, con=engine)

,count
0,821


С 1 января 2000 года вышла 821 книга. 

## Количество обзоров и средняя оценка книг
Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [9]:
# Количество обзоров и средняя оценка книг
query = '''
        SELECT rev_query.title,
               review_count,
               avg_rating
        FROM (SELECT book_id,
                     title,
                     COUNT(review_id) AS review_count
              FROM books
              LEFT OUTER JOIN reviews USING(book_id)
              GROUP BY book_id) AS rev_query
        INNER JOIN
              (SELECT book_id,
                      title,
                      ROUND(AVG(rating), 2) AS avg_rating
              FROM books
              LEFT OUTER JOIN ratings USING(book_id)
              GROUP BY book_id) AS rat_query USING(book_id)
        ORDER BY title;
        '''

pd.io.sql.read_sql(query, con=engine)

,title,review_count,avg_rating
0,'Salem's Lot,2,3.67
1,1 000 Places to See Before You Die,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,1491: New Revelations of the Americas Before C...,2,4.50
4,1776,4,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,Xenocide (Ender's Saga #3),3,3.40
997,Year of Wonders,4,3.20
998,You Suck (A Love Story #2),2,4.50


Сформирована таблица, содержащая количество обзоров и среднюю оценку для каждой книги.

## Издательство, выпустившее наибольшее число книг
Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [10]:
# Издательство, выпустившее наибольшее число книг
query = '''
        SELECT publisher,
               COUNT(book_id) AS books_published
        FROM publishers AS p
        INNER JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE num_pages > 50
        GROUP BY p.publisher_id
        ORDER BY COUNT(book_id) DESC
        LIMIT 1;
        '''

pd.io.sql.read_sql(query, con=engine)

,publisher,books_published
0,Penguin Books,42


Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books.

## Автор с самой высокой средней оценкой книг
Определим автора с самой высокой средней оценкой книг, имеющих от 50 оценок.


In [11]:
# Автор с самой высокой средней оценикой книг
query = '''
        SELECT author,
        ROUND(AVG(avg_book_rating), 2) AS avg_books_rating
        FROM (SELECT author_id, b.book_id, AVG(rating) AS avg_book_rating
              FROM books AS b
              LEFT OUTER JOIN ratings AS r ON b.book_id = r.book_id
              GROUP BY b.book_id
              HAVING COUNT(rating) > 50) AS t
        LEFT OUTER JOIN authors AS a ON t.author_id = a.author_id
        GROUP BY author
        ORDER BY AVG(avg_book_rating) DESC
        LIMIT 1;
        '''

pd.io.sql.read_sql(query, con=engine)

,author,avg_books_rating
0,J.K. Rowling/Mary GrandPré,4.28


Самая высокая средняя оценка книг, имеющих от 50 оценок, у авторов J.K. Rowling/Mary GrandPré.

## Среднее количество обзоров от пользователей
Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
# Среднее количество обзоров от пользователей
query = '''
        SELECT ROUND(AVG(review_count)) AS avg_review_count
        FROM (SELECT username,
              COUNT(review_id) AS review_count
              FROM reviews
              WHERE username in (SELECT username
                                 FROM ratings
                                 GROUP BY username
                                 HAVING COUNT(rating_id) > 50)
              GROUP BY username) AS rc;
        '''

pd.io.sql.read_sql(query, con=engine)

,avg_review_count
0,24.0


В среднем пользователи, которые поставили больше 50 оценок, пишут 24 обзора.